In [28]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
DESCRIPTION

Get mean summer stats for regions.

"""

# Import modules
import numpy as np
import glob
import pandas as pd
import netCDF4
import xarray as xr
from skimage import filters
from skimage import morphology
import scipy.ndimage as ndimage
from scipy import interpolate

# Define path name
path = '/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/'

# Define coordinate system
df_crs = 'EPSG:4326'

# Define cloud data
cloud_data = xr.open_dataset(path + 'final_climatologies.nc')

# Define temperature data
temp_data = xr.open_dataset(path + 'temp_albedo_summer_climatologies.nc')

In [15]:
# Define some functions
def new_linregress(x, y):
    # Wrapper around scipy linregress to use in apply_ufunc
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    return np.array([slope, intercept, r_value, p_value, std_err])


def remove_pepper(array, threshold):
    
    """ 
    Function to remove peppper effects
    
    """
    
    # Convert to binary
    array_bool = np.isfinite(array).astype(int)
    
    # Label
    id_regions = morphology.label(array_bool, background = 0, connectivity = 2)
    
    # Number of regions
    num_ids = id_regions.max()
    
    # Compute region sizes in pixels
    id_sizes = np.array(ndimage.sum(array_bool, id_regions, range(num_ids + 1)))
    
    # Filter out regions smaller than threshold
    area_mask = (id_sizes < threshold)
    array[area_mask[id_regions]] = np.nan
    
    return array

def remove_salt(array, threshold):
    
    """
    Function to remove salt effects
    
    """
    
    # Convert to binary
    array_bool = np.isnan(array).astype(int)
    
    # Label
    id_regions = morphology.label(array_bool, background = 0, connectivity = 2)
    
    # Number of regions
    num_ids = id_regions.max()
    
    # Compute region sizes in pixels
    id_sizes = np.array(ndimage.sum(array_bool, id_regions, range(num_ids + 1)))
    
    # Assign regions larger than the threshold a value of 1
    area_mask = (id_sizes > threshold)

    array[area_mask[id_regions]] = 1
    
    return array

def linear_interpolate(array):
    
    """
    Function to linearly intepolate NaNs in array
    
    """
    masked_array = np.ma.masked_invalid(array)

    x = np.arange(0, masked_array.shape[1])
    y = np.arange(0, masked_array.shape[0])
    xx, yy = np.meshgrid(x, y)

    #get only the valid values
    x1 = xx[~masked_array.mask]
    y1 = yy[~masked_array.mask]

    newarr = masked_array[~masked_array.mask]

    linear = interpolate.griddata((x1, y1), newarr.ravel(),
                              (xx, yy), method='nearest')
    
    return linear

In [26]:
###############################################################################
# Compute linear relationship between albedo and air temperature for every pixel
###############################################################################
xr_stats = xr.apply_ufunc(new_linregress, temp_data['t2m'], 
                          cloud_data['cloudiness'],
                          input_core_dims=[['z'], ['z']],
                          output_core_dims=[["parameter"]],
                          vectorize=True,
                          dask="parallelized",
                          output_dtypes=['float64'],
                          output_sizes={"parameter": 5})

/var/folders/xj/5ps5mr8d5ysbd2mxxqjg3k800000gq/T/ipykernel_4235/866598012.py:4: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  xr_stats = xr.apply_ufunc(new_linregress, temp_data['t2m'],


In [48]:
###############################################################################
# Do filtering
###############################################################################

# Copy 
crop = np.copy(xr_stats)

In [51]:
###############################################################################
# Save 1 km dataset to NetCDF
###############################################################################
dataset = netCDF4.Dataset(path + 'empirical_cloud_model.nc', 
                          'w', format='NETCDF4_CLASSIC')
print('Creating... %s' % path + 'empirical_cloud_model.nc')
dataset.Title = "Slopes and intercepts for temperature vs. cloudiness relationship"
import time
dataset.History = "Created " + time.ctime(time.time())
dataset.Projection = "WGS 84"
dataset.Reference = "Ryan, J. C., Smith. L. C., Cooley, S. W., and Pearson, B. (in review), Emerging importance of clouds for Greenland Ice Sheet energy balance and meltwater production."
dataset.Contact = "jryan4@uoregon.edu"
    
# Create new dimensions
lat_dim = dataset.createDimension('y', linear_inter.shape[0])
lon_dim = dataset.createDimension('x', linear_inter.shape[1])

# Define variable types
Y = dataset.createVariable('latitude', np.float32, ('y','x'))
X = dataset.createVariable('longitude', np.float32, ('y','x'))
    
# Define units
Y.units = "degrees"
X.units = "degrees"
   
# Create the actual 3D variable
slope_nc = dataset.createVariable('slope', np.float32, ('y','x'))
inter_nc = dataset.createVariable('intercept', np.float32, ('y','x'))
r_value_nc = dataset.createVariable('r_value', np.float32, ('y','x'))
p_vaue_nc = dataset.createVariable('p_value', np.float32, ('y','x'))
std_err_nc = dataset.createVariable('std_err', np.float32, ('y','x'))
temp_mean_nc = dataset.createVariable('mean_temp', np.float32, ('y','x'))

# Write data to layers
Y[:] = temp_data['latitude'].values
X[:] = temp_data['longitude'].values
slope_nc[:] = crop[:,:,0]
inter_nc[:] = crop[:,:,1]
r_value_nc[:] = crop[:,:,2]
p_vaue_nc[:] = crop[:,:,3]
std_err_nc[:] = crop[:,:,4]
temp_mean_nc[:] = np.mean(temp_data['t2m'], axis=2)

print('Writing data to %s' % path + 'empirical_cloud_model.nc')
    
# Close dataset
dataset.close()

Creating... /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/empirical_cloud_model.nc
Writing data to /Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/empirical_cloud_model.nc
